In [1]:
import random as rdn
from copy import deepcopy 

In [2]:
class formiga:
    def __init__(self, quantidadeRainhas, evaporacao):
        self.iteracao = 0
        self.custo = quantidadeRainhas
        self.rainhas = quantidadeRainhas
        self.valorEvaporacao = evaporacao
        
        self.caminho = [i for i in range(quantidadeRainhas)] 
        self.matrizFeromonio = [[1 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        self.matrizProbabilidade = [[0 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        
    def caminhoInicial(self):
        rdn.shuffle(self.caminho)
    
    def geraMatrizRainhas(self):
        self.matrizRainhas = [[0 for i in range(self.rainhas)] for j in range(self.rainhas)]
        
        for l, c in enumerate(self.caminho):
            self.matrizRainhas[l][c] = 1
    
    def calcularCusto(self):
        dicionario = {}
        for i in self.caminho:
            if i in dicionario:
                dicionario[i] = dicionario[i] + 1
            else:
                dicionario[i] = 1
            
        self.custo = int(self.custoDiagonal() / 2) + self.custoColuna()
    
    def maiorValorMatrizFeromonio(self):
        maiorValor = 0
        
        for linha in self.matrizFeromonio:
            maiorValorLinha = max(linha)
            if maiorValorLinha > maiorValor:
                maiorValor = maiorValorLinha
        
        return maiorValor
    
    
    def evaporarFeromonio(self):
        for linha in range(self.rainhas):
            for coluna in range(self.rainhas):
                if (coluna != self.caminho[linha]):
                    self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] - self.matrizFeromonio[linha][coluna] * self.valorEvaporacao, 3)
    
    def custoDiagonal(self):
        somaDiagonal = 0 
        
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):

                if self.caminho[i]+(j-i) < len(self.caminho):
                    if self.caminho[i+(j-i)] == self.caminho[i]+(j-i):
                        somaDiagonal += 1
                if self.caminho[i]-(j-i) >= 0:
                    if self.caminho[i+(j-i)] == self.caminho[i]-(j-i):
                        somaDiagonal += 1

            for j in range(0,i):
                if self.caminho[i]+(j+1) < len(self.caminho):
                    if self.caminho[i-(j+1)] == self.caminho[i]+(j+1):
                        somaDiagonal += 1
                if self.caminho[i]-(j+1) >= 0:
                    if self.caminho[i-(j+1)] == self.caminho[i]-(j+1):
                        somaDiagonal += 1
        
        return somaDiagonal
    
    def custoColuna(self):
        custo = 0
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):
                if self.caminho[i] == self.caminho[j]:
                    custo += 1
        return custo
    
    def calcularMatrizFeromonio(self):
        self.calcularCusto()
        
        if (self.custo == 0):
            return self.caminho

        feromonio = round(1 / self.custo, 3) * 0.4
        
        self.evaporarFeromonio()
        
        for linha, coluna in enumerate(self.caminho):
            self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] + feromonio, 3)
                
    def calcularMatrizProbabilidade(self):
        somas = [0 for i in range(self.rainhas)]
            
        for linha in self.matrizFeromonio:    
            for indice, coluna in enumerate(linha):
                somas[indice] += coluna

        for indiceLinha, linha in enumerate(self.matrizFeromonio):  
            for indiceColuna, coluna in enumerate(linha):
                self.matrizProbabilidade[indiceLinha][indiceColuna] = round(coluna / somas[indiceColuna], 3)
                              
    def novoCaminhoFormiga(self):
        i = 0

        for line in self.matrizProbabilidade:
            roleta = [0 for n in line]
            somaPeso = sum(line)
            sorteio = rdn.random()
            
            prob = [x*100/somaPeso for x in line]
            
            roleta[0] = (0, prob[0])
            for x in range(1, len(prob)):
                roleta[x] = (roleta[x-1][1], roleta[x-1][1] + prob[x])

            self.caminho[i] = self.escolherPosicaoArbitraria(roleta, sorteio)
            i += 1
    
    def escolherPosicaoArbitraria(self, roleta, sorteio):
        sorteio *= 100
        posicao = 0
        for minimo, maximo in roleta:
            
            if sorteio >= minimo and sorteio < maximo:
                return posicao
            
            posicao += 1
        
        return posicao
    
    def atualizarMatrizFeromonio(self, novaMatrizFeromonio):
        self.matrizFeromonio = novaMatrizFeromonio
    
    def debug(self):
        print('Iteracao', self.iteracao)
        print('Custo', self.custo)
        
        print('Rainhas')
        printMatriz(self.matrizRainhas)
        
        print('Feromonios')
        printMatriz(self.matrizFeromonio)
        
        print('Probabilidade')
        printMatriz(self.matrizProbabilidade)
    
    def prepararBusca(self):
        self.caminhoInicial()
                
    def encontrarUmCaminho(self):
        self.geraMatrizRainhas()
        self.calcularMatrizFeromonio()
        self.calcularMatrizProbabilidade()
        self.novoCaminhoFormiga()
        
    def retornarConflitos(self):
        return self.custo
    
    def retornarMatrizFeromonio(self):
        return self.matrizFeromonio

In [3]:
def printMatriz(matriz):
    for linha in matriz:
        for coluna in linha:
            print(coluna, end='  ')
        print()
    print()

In [4]:
def menorValorMatrizFeromonio(matrizFeromonio):
    menorValor = matrizFeromonio[0][0]
    for i in range(len(matrizFeromonio)):
        for j in range(len(matrizFeromonio)):
            if matrizFeromonio[i][j] < menorValor:
                menorValor = matrizFeromonio[i][j]
    
    return menorValor

In [5]:
def somaMatrizesFeromonio(matrizesFeromonio, rainhas, formigas):
    matrizFinal = [[0 for i in range(rainhas)] for j in range(rainhas)]
    
    for matriz in matrizesFeromonio:
        matrizFinal = somaMatriz(matrizFinal, matriz)
    
    maiorValorFeromonio = menorValorMatrizFeromonio(matrizFinal)
    for i in range(rainhas):
        for j in range(rainhas):
            matrizFinal[i][j] /= maiorValorFeromonio
            
    return matrizFinal

In [6]:
def somaMatriz(matrizAlvo, matrizFonte):
    ordem = len(matrizAlvo)
    for i in range(ordem):
        for j in range(ordem):
            matrizAlvo[i][j] += matrizFonte[i][j]
            
    return matrizAlvo

In [7]:
import random
import multiprocessing as mp
import os
from multiprocessing.pool import ThreadPool

In [8]:
def main(rainhas, numFormigas, valorEvaporacao):
    conflito = rainhas
    formigas = []
    matrizFeromonioCompartilhada = [[1 for i in range(rainhas)] for j in range(rainhas)]
    
    for i in range(numFormigas):
        formigas.append(formiga(rainhas, valorEvaporacao))
    
    for formigaIndividual in formigas:
        formigaIndividual.prepararBusca()
        
    while conflito > 0:
        conflitos = []
        matrizesFeromonio = []
        
        for formigaIndividual in formigas:
            formigaIndividual.atualizarMatrizFeromonio(deepcopy(matrizFeromonioCompartilhada))
            formigaIndividual.encontrarUmCaminho()
            
            pool = ThreadPool(processes = 2)
            async_call = pool.apply_async(formigaIndividual.encontrarUmCaminho, ())
            async_call.get()

            conflitos.append(formigaIndividual.retornarConflitos())
            matrizesFeromonio.append(formigaIndividual.retornarMatrizFeromonio())
        
        conflito = min(conflitos)
        matrizFeromonioCompartilhada = somaMatrizesFeromonio(matrizesFeromonio, rainhas, numFormigas)
        
        
    return formigas[conflitos.index(0)]

In [12]:
formigaSolucao = main(8, 10, 0.001)
printMatriz(formigaSolucao.matrizRainhas)
printMatriz(formigaSolucao.matrizFeromonio)

KeyboardInterrupt: 

In [34]:
def mainMultiprocesso(rainhas, quantidadeFormigas):
    formigas = []
    threads = []
    result = None
    
    print('Quantidade de cpu', mp.cpu_count())
    pool = mp.Pool(mp.cpu_count())
    event = mp.Event()

    for novaFormiga in range(quantidadeFormigas):
        formigas.append(formiga(rainhas))

    for novaFormiga in formigas:
        pool.apply_async(novaFormiga.encontrarSolucaoMultiprocesso(event))
    
    pool.close()
    pool.join()
        
    print(result)
    return result

def main(rainhas, valorEvaporacao):
    formigaSolucao = formiga(rainhas, valorEvaporacao)
    return formigaSolucao.encontrarSolucao()

### Teste utilizando Threads e Multiprocessings

In [23]:
import threading, random, multiprocessing
from multiprocessing.pool import ThreadPool

result = 0
quantidadeMultiprocessos = multiprocessing.cpu_count()

def worker(threadID, event):
    values = [i for i in range(10)]
    x = values[0]
    while (not(x == 5) and not(event.is_set())):       
        random.shuffle(values)
        x = values[0]
        print("Value is {} and Worker is {}".format(x, threadID))
        if x == 5:
            event.set()
            print('Worker', threadID)
            global result
            result = x
            return x

if __name__ == '__main__':   
    jobs = []
    # event = threading.Event()
    event = multiprocessing.Event()

    for i in range(5):
#         t = threading.Thread(target=worker, args=(i, event))
        t = multiprocessing.Process(target=worker(multiprocessing.current_process(), event))
#         jobs.append(t)
        t.start()

Value is 2 and Worker is <_MainProcess(MainProcess, started)>
Value is 7 and Worker is <_MainProcess(MainProcess, started)>
Value is 5 and Worker is <_MainProcess(MainProcess, started)>
Worker <_MainProcess(MainProcess, started)>


In [19]:
import multiprocessing as mp
import numpy as np
from time import time

pool = mp.Pool(mp.cpu_count())

# Prepare data
np.random.RandomState(100)
arr = np.random.randint(0, 10, size=[2000, 5])
data = arr.tolist()

results = Queue()

# Step 1: Redefine, to accept `i`, the iteration number
def howmany_within_range2(i, row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return (i, count)


# Step 2: Define callback function to collect the output in `results`
def collect_result(result):
    global results
    results.append(result)


# Step 3: Use loop to parallelize
for i, row in enumerate(data):
    pool.apply_async(howmany_within_range2(i, row, 4, 8))

# Step 4: Close Pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

print(results)

[]


In [32]:
import random
import multiprocessing as mp
import os

print('Quantidade de cpu', mp.cpu_count())
pool = mp.Pool(4)

result = 0

def worker(event, threadID):
    values = [i for i in range(10)]
    x = values[0]
    while (not(x == 5) and not(event.is_set())):       
        random.shuffle(values)
        x = values[0]
        print("Value is {} and Worker is {}".format(x, threadID))
        if x == 5:
            event.set()
            print('Worker', threadID)
            
            global result
            result = x
            
            return x

event = mp.Event()

pool.apply_async(worker(event, os.getpid()))

pool.close()
pool.join()

print(result)

Quantidade de cpu 2
Value is 8 and Worker is 12056
Value is 9 and Worker is 12056
Value is 2 and Worker is 12056
Value is 4 and Worker is 12056
Value is 4 and Worker is 12056
Value is 0 and Worker is 12056
Value is 7 and Worker is 12056
Value is 0 and Worker is 12056
Value is 6 and Worker is 12056
Value is 8 and Worker is 12056
Value is 4 and Worker is 12056
Value is 6 and Worker is 12056
Value is 7 and Worker is 12056
Value is 2 and Worker is 12056
Value is 5 and Worker is 12056
Worker 12056
5
